In [2]:
import shutil

original_dataset_path = "/kaggle/input/medqa-senior/dataset.txt"
destination_path = "/kaggle/working/dataset_copy.txt"
shutil.copyfile(original_dataset_path, destination_path)
dataset_path = destination_path

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_path,
    block_size=128 
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)


trainer.train()
model.save_pretrained("./fine_tuned_model")



2024-04-26 16:28:54.130548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 16:28:54.130674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 16:28:54.276194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)


  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import textwrap
import re

# Load fine-tuned GPT-2 model and tokenizer
model_path = "./fine_tuned_model"  # Path to your fine-tuned model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_path)
pattern = r".*\?[\n\n| ]"

# Define a function to generate responses to questions
def generate_response(question, max_length=100):
    input_text = question
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids,pad_token_id=tokenizer.pad_token_id, 
                            max_length=max_length, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def answer(question):
    response = generate_response(question)
    response = re.split(pattern,response[:response.rfind('.')+1])
    response = response[1].strip()
    return textwrap.fill(response,width = 80)

In [1]:
questions = ["what is diabetes?",
            "what causes diabetes?",
            "What are the symptoms of diabetes?",
            "How is diabetes diagnosed?",
            "What are the potential complications of diabetes, and how can I reduce my risk of developing them?",
            "How often should I monitor my blood sugar levels, and what are the target ranges I should aim for?",
            "What medications or insulin regimen do I need to take, and what are the potential side effects?",
            "Are there any vaccines or screenings that are recommended for people with diabetes?"]
for i in questions:
    print(f"{i}\n{answer(i)}\n")

what is diabetes?
Diabetes is a chronic disease that affects the blood sugar level. It affects
about one in every 100 people.  People with diabetes have trouble controlling
their blood sugar levels. They may have trouble eating, sleeping, and getting
enough rest.  People with diabetes may also have other health problems,
including heart disease, stroke, kidney disease, and certain cancers.

-----------------------------------------------------------------------------

what causes diabetes?
Diabetes is a disease of the blood cells. The cells in your blood are made of
proteins called beta cells. These cells are responsible for producing insulin,
the hormone that helps your body absorb food and keep you healthy.  The body
uses insulin to help regulate blood sugar levels. Insulin helps your body use
glucose from your blood to fuel energy. Insulin also helps your body use fat
from your body to fuel energy.

-----------------------------------------------------------------------------

What 

In [29]:
from nltk.translate.bleu_score import sentence_bleu


split = lambda x: x.split()
reference = ["Diabetes is a chronic condition characterized by high levels of sugar (glucose) in the blood. It occurs when the body either doesn't produce enough insulin or can't effectively use the insulin it produces. Insulin is a hormone that regulates blood sugar. Without proper management, diabetes can lead to serious complications such as heart disease, stroke, kidney damage, and nerve damage.",
            "Diabetes is a long-term health condition where the body struggles to regulate blood sugar levels properly. This happens either because the pancreas doesn't produce enough insulin or because the body's cells don't respond adequately to insulin. Insulin is crucial for controlling blood sugar levels. If left unmanaged, diabetes can result in various serious complications, including heart disease, stroke, kidney issues, and nerve damage.",
            "Diabetes is a metabolic disorder characterized by elevated levels of glucose (sugar) in the blood. It occurs due to insufficient insulin production or the body's inability to effectively use insulin. Insulin is essential for regulating blood sugar levels. Without proper management, diabetes can lead to severe complications like cardiovascular disease, kidney failure, nerve damage, and vision problems.",
            "Diabetes is a persistent health condition impacting blood sugar levels, with approximately one in every 100 individuals affected. Those with diabetes experience challenges in maintaining adequate blood sugar levels necessary for supporting normal blood pressure and heart function.",
            "Diabetes is a chronic disease that affects the blood sugar level. It affects about one in every 100 people.  People with diabetes have trouble getting blood sugar to support normal blood pressure and heart function."]

reference = list(map(split,reference))
question = "What is diabetes?"
candidate = answer(question).split()

score = sentence_bleu(reference,candidate)
print(score)

0.8511926813499732


In [42]:
# Example usage
question = "What is diabetes?"
answer(question)

Response: Diabetes is a chronic disease that affects the blood sugar level. It affects
about one in every 100 people.  People with diabetes have trouble getting enough
blood sugar (sugar) to support normal blood pressure and heart function.


In [39]:
question = "How does diabetes affect blood sugar levels?"
answer(question)

Response: Diabetes affects the body's ability to use sugar as fuel. When blood sugar
levels are too high, the body cannot use glucose properly.


In [44]:
question = "What are the risks of Diabetes?"
answer(question)

Response: Diabetes is a chronic disease that affects the body's ability to use glucose. It
affects the body's ability to use insulin, the hormone that regulates blood
sugar. Insulin helps the body use glucose to fuel the body's cells.  Diabetes
affects about one in every 100 people.  People with diabetes have a higher risk
of developing other types of heart disease, stroke, kidney disease, and certain
cancers.


In [51]:
question = "What is Alzheimer's disease?"
answer(question)

Response: Alzheimer's disease is a progressive brain disease that affects the brain and
body. It is caused by a buildup of plaques and tangles in the brain. The brain
is made up of nerve cells called neurons. The brain is divided into two parts:
the frontal lobe, which controls movement, and the temporal lobe, which controls
thinking and memory. Alzheimer's affects the brain's wiring and connections
between these two parts.


In [60]:
question = "What are the common symptoms of Alzheimer's disease?"
print(generate_response(question))

What are the common symptoms of Alzheimer's disease?

Symptoms of Alzheimer's disease include:

Memory loss

Difficulty concentrating

Difficulty thinking

Difficulty remembering things

Difficulty thinking clearly

Difficulty remembering things

Difficulty remembering things

Difficulty thinking clearly

Difficulty thinking clearly

Difficulty thinking clearly

Difficulty thinking clearly

Difficulty thinking clearly

Difficulty thinking clearly

Difficulty thinking clearly
